# Exercises on Rigid Transforms

In [4]:
# python libraries
import numpy as np
from pydrake.all import RigidTransform, RotationMatrix

# Problem Description
In the lecture, we learned the basics of spatial transformations. In this exercise, you will compute simple rigid transforms applying the rules you have learned in class.

**These are the main steps of the exercise:**
1. Compute rigid transforms of frames in various reference frames.
2. Design grasp pose using spatial transformation

# Exercise on Rigid Transforms

As a brief review, we have covered two rules of spatial transformation in [class](http://manipulation.csail.mit.edu/pick.html#spatial_algebra).

$${^AX^B} {^BX^C} = {^AX^C},$$

$$[^AX^B]^{-1} = {^BX^A}.$$

Note that the rules of transforms are based on rules of transforming positions and rotations listed below. 

Addition of positions in the same frame:
$$^Ap^B_F + ^Bp^C_F = ^Ap^C_F.$$

The additive inverse:
$$^Ap^B_F = - ^Bp^A_F.$$

Rotation of a point:
$$^Ap^B_G = {^GR^F} ^Ap^B_F.$$

Chaining rotations:
$${^AR^B} {^BR^C} = {^AR^C}.$$

Inverse of rotations:
$$[^AR^B]^{-1} = {^BR^A}.$$
      
Applying these rules will yield the same result as the ones computed by the former two rules.

In Drake, you can multiply frames by 
```python
X_AB.multiply(X_BC)
X_AB @ X_BC
```

You may also inverse a rigid transform by the [inverse](https://drake.mit.edu/pydrake/pydrake.math.html?highlight=rigidtransform#pydrake.math.RigidTransform.inverse) method.

```python
X_AB.inverse()
```

Now suppose you have 4 frames, namely, the world frame, frame A, frame B, and frame C defined as below.

-- frame A expressed in the world frame (`X_WA`)

-- frame B expressed in frame A (`X_AB`)

-- frame B expressed in frame C (`X_CB`)

**Calcuate the following transforms by filling your code below in the designated functions.**

(1) `X_WB`, frame B expressed in the world frame

(2) `X_CW`, the world frame expressed in frame C

In [22]:
def compute_X_WB(X_WA, X_AB, X_CB):
    """
    fill your code here
    """
    X_WB = X_WA @ X_AB
    return X_WB

In [6]:
def compute_X_CW(X_WA, X_AB, X_CB):
    """
    fill your code here
    """
    X_CW = X_CB @ X_AB.inverse() @ X_WA.inverse()
    return X_CW

# Design Grasp Pose
The grasp pose is commonly defined in the object frame so that the grasp pose ${^OX^G}$ is independent of the pose of the object. The grasp pose in the world frame can be computed by 

$${{^WX^G} = {}{^W}X^{O}} {^OX^G},$$

where $W$ stands for the world frame and $G$ denotes the grasp frame, following the convention in the textbook. 

You should notice from the visualization below that the gripper frame is different from the world frame. In particular, the +y axis of the gripper frame points vertically downward, and the +z axis of the gripper points backward. This is an important observation for this exercise.

**Now for your exercise, use the gripper and object's orientation from the image to design a grasp pose that satisfy the conditions below**

- **gripper's position should be 0.02 unit distance above the center of the target object in the world frame**
- **gripper's z axis should align with object's x axis**
- **gripper's x axis should align with object's z axis**
- **write grasp pose in the object frame and the world frame**

**Remember that the X-axis is shown in red, Y-axis is in green, and Z-axis in blue.**

<img src="https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/grasp_pose_design.png" width="500">


In [7]:
# Establish the pose of the block expressed in the world frame as shown in the image above.

p0_WO = [0.5, 0.1, 0]  # object in world frame
R0_WO = RotationMatrix.MakeZRotation(-np.pi / 2)
R0_WO = R0_WO.multiply(RotationMatrix.MakeXRotation(np.pi / 2))
X_WO = RigidTransform(R0_WO, p0_WO)

In [100]:
def design_grasp_pose(X_WO):
    """
    fill in our code below
    """
    p_OG = [0, 0.02, 0]
    R_OG = RotationMatrix.MakeXRotation(-np.pi)
    R_OG = R_OG.multiply(RotationMatrix.MakeYRotation(np.pi/2))
    X_OG = RigidTransform(R_OG, p_OG)
    X_WG = X_WO @ X_OG
    return X_OG, X_WG

**When you are done, make sure to run the tests below to make sure everything is working as expected!**

In [103]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_rigid_transforms import TestRigidTransforms

Grader.grade_output([TestRigidTransforms], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 4/4.

Score for Testing X_CW is 1/1.

Score for Testing X_WB is 1/1.

Score for Testing grasp pose is 2/2.


# VERIFICATION IN GRADESCOPE

Run the cells below and copy/paste your answers (rounded to 4 decimal places) into Gradescope.

In [106]:
X_WA = RigidTransform([0.1, 0.2, 0.3])
X_AB = RigidTransform(
    RotationMatrix.MakeXRotation(np.pi / 2), np.array([-0.1, 0.4, 0.5])
)
X_CB = RigidTransform(
    RotationMatrix(np.array([[0, 0, 1], [0, -1, 0], [1, 0, 0]])),
    np.array([0.0, 0.0, 0.4]),
)

## Verification 1: `compute_X_WB`
**Question:** What is the z-position of frame `B` relative to frame `W`

In [109]:
print(f"{compute_X_WB(X_WA, X_AB, X_CB).GetAsMatrix4()[2, 3].item()}")

0.8


## Verification 2: `compute_X_WB`
**Question:** What is the x-position of frame `B` relative to frame `W`

In [112]:
print(f"{compute_X_CW(X_WA, X_AB, X_CB).GetAsMatrix4()[0, 3].item()}")

0.6
